In [1]:
import scanpy as sc
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix

In [2]:
FILE = "datasets/10XGenomics/Targeted_SC3v3_Human_Glioblastoma_Neuroscience_filtered_feature_bc_matrix.h5"

In [3]:
adata = sc.read_10x_h5(FILE)
X = adata.X.todense()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [4]:
B = lambda i: len(bin(i)) - 2
def cell(i):
    return np.array(X[i, :]).flatten()

In [5]:
ncells = X.shape[0]
cells = []
for i in range(ncells):
    cells.append(cell(i))

In [6]:
c = Counter(np.array(cells).flatten())
del c[0]

In [7]:
N = np.array(X).flatten().shape[0]
O = c[1]

i = 1
body = []
while i in c:
    body.append(c[i])
    i += 1
K = i - 1

T = 0
T1 = []
T2 = []
M = int(max(c))
for j in range(i, M + 1):
    if j not in c:
        continue
    T += 1
    T1.append(j)
    T2.append(c[j])

S = max(T2)

In [8]:
Xf = np.array(cells).flatten()
d = defaultdict(list)
for a, b in enumerate(Xf):
    d[b].append(a)

In [9]:
e = {}
for k, v in d.items():
    if v:
        e[k] = [v[0]] + [v[i + 1] - v[i] for i in range(len(v) - 1)]

In [10]:
R = []
for i in range(101, M + 1):
    if i in d:
        for j in d[i]:
            R.append(j)

In [17]:
def write_bytes(handle, bits, bytes=4):
    handle.write(int(bits[::-1], 2).to_bytes(bytes, 'little'))
    return bytes * 4

def write_int(handle, value, bytes=4):
    handle.write(value.to_bytes(bytes, 'little'))
    return bytes * 4

def write_array(handle, array):
    bits = len(bin(max(array))) - 2
    write_int(handle, len(array))
    write_int(handle, bits)

    # print(len(array), bits, (len(array) * 23) - (len(array) * bits + 2 * 32 * 8))
    curr_str = ""
    for elem in array:
        s = bin(elem)[2:].zfill(bits)
        for char in s:
            curr_str += char
            if len(curr_str) == 32:
                write_bytes(handle, curr_str)
                curr_str = ""
    if curr_str:
        write_bytes(handle, curr_str)

    return len(array) * bits + 2 * 32 * 8

In [18]:
with open("10x.conq", "wb") as k:
    total = 0
    total += write_int(k, X.shape[1])
    for i in range(1, 101):
        total += write_array(k, e[i])
    total += write_array(k, R)
    total += write_array(k, body)
    total += write_array(k, T1)
    total += write_array(k, T2)
    print(total)

19953572


In [40]:
def read_bytes(handle, bytes=4):
    byte_data = handle.read(bytes)
    value = int.from_bytes(byte_data, 'little')
    return bin(value)[2:].zfill(32)[::-1]

def read_int(handle, bytes=4):
    return int.from_bytes(handle.read(bytes), 'little')

def read_array(handle):
    array_length = read_int(handle)
    bits = read_int(handle)
    
    array = []
    curr_str = ""
    remaining_bits = array_length * bits
    
    while remaining_bits > 0:
        chunk = read_bytes(handle)
        if chunk is None:
            break
        curr_str += chunk
        
        while len(curr_str) >= bits and len(array) < array_length:
            element_bits = curr_str[:bits]
            curr_str = curr_str[bits:]
            array.append(int(element_bits, 2))
            remaining_bits -= bits

    return array

In [53]:
def flat_to_square(val, ncols):
    return (val // ncols, val % ncols)

def conq_to_csr(handle):
    ncols = read_int(handle)
    e = {}
    for i in range(1, 101):
        e[i] = read_array(handle)
    R = read_array(handle)
    body = read_array(handle)
    T1 = read_array(handle)
    T2 = read_array(handle)

    rows = []
    cols = []
    data = []

    for k, v in e.items():
        c = 0
        for i in v:
            c += i
            data.append(k)
            row, col = flat_to_square(c, ncols)
            rows.append(row)
            cols.append(col)
    i = 0
    for v, n in enumerate(body[100:], start=101):
        for _ in range(n):
            row, col = flat_to_square(R[i], ncols)
            data.append(v)
            rows.append(row)
            cols.append(col)
            i += 1

    for n, m in zip(T1, T2):
        for _ in range(m):
            row, col = flat_to_square(R[i], ncols)
            data.append(n)
            rows.append(row)
            cols.append(col)
            i += 1        
    
    return csr_matrix((data, (rows, cols)))

a = conq_to_csr(X.shape[1], e, R, body, T1, T2)
a

<5697x1186 sparse matrix of type '<class 'numpy.float64'>'
	with 1850700 stored elements in Compressed Sparse Row format>

In [58]:
T1

[484,
 485,
 486,
 487,
 489,
 490,
 492,
 493,
 494,
 495,
 496,
 497,
 498,
 499,
 500,
 501,
 502,
 503,
 504,
 505,
 506,
 507,
 509,
 510,
 511,
 512,
 513,
 515,
 516,
 517,
 518,
 519,
 520,
 521,
 522,
 523,
 524,
 525,
 526,
 527,
 528,
 529,
 530,
 531,
 532,
 533,
 535,
 536,
 538,
 539,
 540,
 541,
 542,
 543,
 544,
 546,
 547,
 548,
 549,
 550,
 551,
 552,
 553,
 554,
 555,
 557,
 558,
 559,
 560,
 561,
 562,
 563,
 564,
 565,
 566,
 567,
 568,
 569,
 570,
 571,
 572,
 573,
 574,
 575,
 576,
 577,
 578,
 579,
 580,
 581,
 582,
 583,
 584,
 585,
 586,
 587,
 588,
 589,
 590,
 591,
 592,
 593,
 594,
 595,
 596,
 597,
 598,
 599,
 600,
 602,
 604,
 605,
 606,
 608,
 609,
 610,
 611,
 612,
 613,
 614,
 615,
 616,
 617,
 618,
 619,
 621,
 622,
 623,
 624,
 625,
 626,
 627,
 628,
 630,
 631,
 632,
 633,
 634,
 635,
 636,
 637,
 639,
 642,
 644,
 645,
 646,
 647,
 648,
 649,
 650,
 651,
 652,
 653,
 654,
 655,
 656,
 658,
 659,
 660,
 661,
 662,
 663,
 664,
 665,
 666,
 668,
 669

In [50]:
b = csr_matrix(X)

In [54]:
(a != b).nonzero()

(array([1988], dtype=int32), array([79], dtype=int32))

In [55]:
a[1988, 79]

0.0

In [56]:
b[1988, 79]

2551.0